In [1]:
import Grisette

# Solver-Aided Programming with Grisette

## Introduction

Grisette is a powerful, battery-included, Haskell library that enables solver-aided programming.
It allows you to build program reasoning tools with off-the-shelf constraint solvers by introducing symbolic values and performing symbolic execution.

In this tutorial, you will learn about

- how to use symbolic types in Grisette, and
- how to call a solver to reason about symbolic values, and
- how to construct a small verifier in Grisette.

We will also provide a small exercise on building a sudoku solver with Grisette.

Please make sure that you have `z3` (https://github.com/Z3Prover/z3) installed and accessible in `$PATH`.

Note that some inline `code blocks` have links to the documentation. It's possible that they are not rendered in a visible way as a link in jupyter notebooks.

## Symbolic Types

Let's start with a simple Haskell code snippet that processes two input `a` and `b`, and returns `a + a + b`.

In [2]:
conFun :: Integer -> Integer -> Integer
conFun a b = a + a + b

When applied to concrete values like 10 and 20, `conFun` will return the concrete result.

In [3]:
conFun 10 20

40

Grisette extends the programming model with symbolic types. In the following code, the [`SymInteger`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymInteger) represents a symbolic integer, which can be a ***symbolic term*** over ***symbolic constants***.
A ***symbolic constant*** is a placeholder for a constant value and can be created using its name with the [`OverloadedStrings`](https://ghc.gitlab.haskell.org/ghc/doc/users_guide/exts/overloaded_strings.html) extension.

Here is an example of the symbolic function that constructs a symbolic term.

In [4]:
symFun :: SymInteger -> SymInteger -> SymInteger
symFun a b = a + a + b

When applied to symbolic values, `symFun` will return a symbolic result. The result of `symFun "a" "b"` is the symbolic term `(+ (+ a a) b)`. Note that the symbolic constants with the same name are the same symbolic constant.

In [5]:
{-# LANGUAGE OverloadedStrings #-}
symFun "a" "b"

(+ (+ a a) b)

Being symbolic does not mean that the type can only hold symbolic constants or terms over symbolic constants.
Concrete values can also be expressed with symbolic types, and Grisette will try to reduce the term when applying symbolic functions to concrete values (with symbolic types).

In [6]:
symFun 1 "b"

(+ 2 b)

Grisette currently provides support for primitive symbolic types for the following types.
You can checkout the documentation for details.

- Booleans ([`SymBool`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymBool)).
- Integers ([`SymInteger`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymInteger)).
- Symbolic signed ([`SymIntN`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymIntN)) and unsigned ([`SymWordN`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymIntN)) bit-vectors.
- Arbitrary precision IEEE-754 floating point numbers ([`SymFP`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymFP)).
- Uninterpreted functions ([`=~>`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:-61--126--62-), [`-~>`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:-45--126--62-)).

## Solver-Aided Reasoning

The power of symbolic values lies in their ability to be translated to off-the-shelf solvers for reasoning about programs.
Let's consider the following question:

- For what input `a` is the result of `symFun a 20` equal to 40?

To answer this, we construct a symbolic boolean term asserting that `symFun a 20` is equal to 40.

In [7]:
a :: SymInteger
a = "a"
print $ symFun a 20 .== 40
Right model <- solve z3 $ symFun a 20 .== 40
print model

(= (+ a a) 20)

Model {a -> 10 :: Integer}

Note that we use [`.==`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:.-61--61-) (symbolic equality) instead of [`==`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:-61--61-) to construct the symbolic term.
Grisette provides many symbolic operators like this. They are usually named with the prefix `.` (for operators) or `sym` (for functions).

The [`solve`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:solve) function sends the term to the solver (in this case, the Z3 solver).
The solver treats all the symbolic constants in the boolean term as existentially quantified and tries to find assignments for them.
It will then return either a [***model***](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:model) or a failure.
Here, a ***model*** is a mapping from symbolic constants to concrete constants.
If we substitute the symbolic constants in the boolean term being solved, the term will evaluate to true.

We can then use the model to evaluate a symbolic term and perform the substitution, with [`evalSymToCon`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:evalSymToCon).

In [8]:
print (evalSymToCon model a :: Integer)
print (evalSymToCon model (symFun a 20) :: Integer)
print (evalSymToCon model (symFun a 20 .== 40) :: Bool)

10

40

True

Sometimes, the solver may not find a model or may conclude that no such model exists. For example, if we ask the solver whether there exists an integer `a` such that `symFun a 20` is equal to 39, the solver will conclude that the constraint cannot be satisfied:

In [9]:
r <- solve z3 $ symFun a 20 .== 39
print r

Left Unsat

The result, in this case, would be `Left Unsat`, indicating that the constraint is unsatisfiable.

## A Small Expression Equivalence Verifier

Now that we have a basic understanding of Grisette's symbolic types and solver-aided reasoning, let's dive into building a practical application: an expression equivalence verifier. This tool can be particularly useful when developing an optimizing compiler that performs term rewrites. By verifying the equivalence of expressions before and after the rewrite, you can ensure the correctness of your optimization rules.

We'll start by defining a small language for our expressions:

```
E -> I int
   | B bool
   | Add E E
   | Mul E E
   | Eq E E
```

Using GADTs (Generalized Algebraic Data Types), we can define well-typed instances of our expression language as follows:

(If you aren't familiar with GADTs, you may refer to https://en.wikibooks.org/wiki/Haskell/GADT. Our example is also adapted from the expression language defined there.)

In [10]:
{-# LANGUAGE GADTs #-}
data Expr a where
  I :: SymInteger -> Expr SymInteger
  B :: SymBool -> Expr SymBool
  Add :: Expr SymInteger -> Expr SymInteger -> Expr SymInteger
  Mul :: Expr SymInteger -> Expr SymInteger -> Expr SymInteger
  Eq :: SymEq a => Expr a -> Expr a -> Expr SymBool

Note the [`SymEq`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:SymEq) constraint in the `Eq` constructor. This is a type class provided by Grisette that enables the use of [`.==`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:.-61--61-) or [`./=`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:.-47--61-) operators for symbolic equality and inequality comparisons.

Next, let's define an evaluation function for our expression language:

In [11]:
eval :: Expr a -> a
eval (I n) = n
eval (B b) = b
eval (Add e1 e2) = eval e1 + eval e2
eval (Mul e1 e2) = eval e1 * eval e2
eval (Eq e1 e2) = eval e1 .== eval e2

With our language and evaluation function in place, we can now build the equivalence verifier. To prove that two expressions are equivalent, we need to show that the following property $p_1$ holds. Here, $\mathrm{consts}(e)$ is the set of all symbolic constants in the expression $e$.

$p_1=\forall~\mathrm{constant}\in \mathrm{consts}(e_1), \mathrm{consts}(e_2). \mathrm{eval}(e_1) = \mathrm{eval}(e_2)$

However, solvers typically struggle with efficiently handling universally quantified formulas. To circumvent this, we can transform $p_1$ into an existentially quantified formula $p_2$, such that $p_2$ is unsatisfiable implies that $p_1$ is a tautology:

$p_2=\exists~\mathrm{constant}\in \mathrm{consts}(e_1), \mathrm{consts}(e_2). \mathrm{eval}(e_1) \neq\mathrm{eval}(e_2)$

Here's the implementation of the equivalence verifier.

In [12]:
verifyEquivalent :: (EvalSym a, SymEq a, Show a) => Expr a -> Expr a -> IO ()
verifyEquivalent e1 e2 = do
  res <- solve z3 $ eval e1 ./= eval e2
  case res of
    Left Unsat -> putStrLn "The two expressions are equivalent"
    Left err -> putStrLn $ "The solver returned unexpected response: " <> show err
    Right model -> do
      putStrLn "The two expressions are not equivalent, under the model:"
      print model
      putStrLn $ "lhs evaluates to: " <> show (evalSym False model $ eval e1)
      putStrLn $ "rhs evaluates to: " <> show (evalSym False model $ eval e2)

The [`EvalSym`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:EvalSym) type class used here is another Grisette type class that enables the substitution of symbolic constants with concrete values from a model. [`evalSym`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:evalSym)`False` substitutes only the symbolic constants present in the model, while [`evalSym`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:evalSym)`True` assigns default concrete values to any unmentioned constants.

Let's put our verifier to the test! First, we'll prove the distributive property of multiplication over addition:

In [13]:
verifyEquivalent
  (Mul (I "a") (Add (I "b") (I "c")))
  (Add (Mul (I "a") (I "b")) (Mul (I "a") (I "c")))

The two expressions are equivalent

Next, we'll attempt to prove an incorrect equivalence:

In [14]:
verifyEquivalent
  (Add (I "a") (Mul (I "b") (I "c")))
  (Mul (Add (I "a") (I "b")) (Add (I "a") (I "c")))

The two expressions are not equivalent, under the model:
Model {a -> -3 :: Integer, b -> 2 :: Integer, c -> 7 :: Integer}
lhs evaluates to: 11
rhs evaluates to: -4

As expected, the solver will provide a counterexample model demonstrating that the expressions are not equivalent.

Finally, let's verify a simple boolean equivalence ([con](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:con) converts a concrete boolean value to a symbolic boolean term, or you can use [true](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:true) to create concrete or symbolic true terms):

In [15]:
verifyEquivalent (Eq (I "a") (I "a")) (B $ con True)
verifyEquivalent (Eq (I "a") (I "a")) (B true)

The two expressions are equivalent

The two expressions are equivalent

## Exercise: sudoku solver

Now let's do some exercise. We will build a sudoku solver with Grisette.

From [wikipedia](https://en.wikipedia.org/wiki/Sudoku), a sudoku is a number-placement puzzle.
Its objective is to fill a 9 x 9 grid with digits so that each column, each row, and each of the nine 3 x 3 regions that compose the grid contains all the digits from 1 to 9.

The following shows a typical sudoku puzzle:

<img src="./_img/sudoku.svg" alt="drawing" style="width:200px; margin-left: auto; margin-right: auto">

And the solution to it is:

<img src="./_img/sudoku_solution.svg" alt="drawing" style="width:200px; margin-left: auto; margin-right: auto">

Your task is to implement a symbolic sudoku checker and use the z3 solver to turn the checker into a sudoku solver.
You can search for TODOs for some tasks for you, and we will provide answers to them at the end of this tutorial.

### Concrete sudoku ***checker***

We first provide the concrete sudoku ***checker*** as a reference. It can check whether a sudoku board is valid, but cannot solve sudoku puzzles.

A sudoku board can be modeled as a nested list, as shown below.

In [16]:
type Row = [Integer]
type Board = [Row]

The following then shows some sudoku boards. The first one is valid.
The second one is invalid because it contains 10, which isn't a valid number.
The third one is also invalid because the first column contains two 6s, which violates the rules of sudoku.

In [17]:
board :: Board
board =
  [ [7, 3, 4, 1, 6, 2, 9, 8, 5],
    [6, 8, 5, 4, 7, 9, 3, 2, 1],
    [2, 1, 9, 5, 3, 8, 6, 4, 7],
    [5, 6, 8, 9, 1, 3, 2, 7, 4],
    [3, 4, 2, 6, 8, 7, 1, 5, 9],
    [1, 9, 7, 2, 5, 4, 8, 3, 6],
    [8, 5, 1, 7, 2, 6, 4, 9, 3],
    [9, 2, 6, 3, 4, 5, 7, 1, 8],
    [4, 7, 3, 8, 9, 1, 5, 6, 2]
  ]

boardOutOfBounds :: Board
boardOutOfBounds =
  [ [10, 3, 4, 1, 6, 2, 9, 8, 5],
    [6, 8, 5, 4, 7, 9, 3, 2, 1],
    [2, 1, 9, 5, 3, 8, 6, 4, 7],
    [5, 6, 8, 9, 1, 3, 2, 7, 4],
    [3, 4, 2, 6, 8, 7, 1, 5, 9],
    [1, 9, 7, 2, 5, 4, 8, 3, 6],
    [8, 5, 1, 7, 2, 6, 4, 9, 3],
    [9, 2, 6, 3, 4, 5, 7, 1, 8],
    [4, 7, 3, 8, 9, 1, 5, 6, 2]
  ]

boardOverlapping :: Board
boardOverlapping =
  [ [6, 3, 4, 1, 6, 2, 9, 8, 5],
    [6, 8, 5, 4, 7, 9, 3, 2, 1],
    [2, 1, 9, 5, 3, 8, 6, 4, 7],
    [5, 6, 8, 9, 1, 3, 2, 7, 4],
    [3, 4, 2, 6, 8, 7, 1, 5, 9],
    [1, 9, 7, 2, 5, 4, 8, 3, 6],
    [8, 5, 1, 7, 2, 6, 4, 9, 3],
    [9, 2, 6, 3, 4, 5, 7, 1, 8],
    [4, 7, 3, 8, 9, 1, 5, 6, 2]
  ]

We can validate a Sudoku board by ensuring that all rows, columns, and 3x3 regions contain distinct numbers from 1 to 9.

In [18]:
import Data.List (transpose)

check :: Row -> Bool
check xs = distinct xs && inbound xs
  where
    distinct :: Row -> Bool
    distinct [] = True
    distinct (x : xs1) = notElem x xs1 && distinct xs1
    inbound :: Row -> Bool
    inbound = all (\x -> x >= 1 && x <= 9)

columns = transpose
regionAt rows x y = [rows !! i !! j | i <- [x .. x + 2], j <- [y .. y + 2]]
regions rows = [regionAt rows (i * 3) (j * 3) | i <- [0 .. 2], j <- [0 .. 2]]
toCheck rows = rows ++ columns rows ++ regions rows

checkBoard :: Board -> Bool
checkBoard rows = all check (toCheck rows)

print $ checkBoard board
print $ checkBoard boardOutOfBounds
print $ checkBoard boardOverlapping

True

False

False

### Symbolic sudoku ***checker***

Suppose we want to build a sudoku solver.
Without constraint solving, we typically need to implement some sort of searching algorithm to search within the possible solution space.
This isn't very hard, but we will show that constraint solvers can make it even simpler by turning a ***symbolic checker*** into a ***puzzle solver***, and we will see that with Grisette, a symbolic checker looks very similar to a concrete checker.

First, we define our symbolic board type. We can do this by replacing `Integer` with `SymInteger` in the definition.

In [19]:
type SymRow = [SymInteger]
type SymBoard = [SymRow]

You can then implement a symbolic checker, using the concrete checker as a reference.

Grisette provides symbolic counterparts for many Haskell library functions.
The following functions ([`true`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:true), [`symNotElem`](https://hackage.haskell.org/package/grisette/docs/Grisette-Lib-Data-List.html#v:symNotElem), and [`symAll`](https://hackage.haskell.org/package/grisette/docs/Grisette-Lib-Data-Foldable.html#v:symAll)) might be useful in your symbolic checker.

In [20]:
:t true
:t symNotElem
:t symAll

true :: forall b. LogicalOp b => b

symNotElem :: forall (t :: * -> *) a. (Foldable t, SymEq a) => a -> t a -> SymBool

symAll :: forall (t :: * -> *) a. Foldable t => (a -> SymBool) -> t a -> SymBool

In [21]:
symCheck :: SymRow -> SymBool
-- TODO: Implement the symCheck function. You can use the check function as a reference
symCheck = TODO

checkSymBoard :: SymBoard -> SymBool
checkSymBoard rows = symAll symCheck (toCheck rows)

: 

### Sudoku puzzles

The following code generates some symbolic sudoku puzzles.

Here, we define a `genSymBoard` function.
It generates a symbolic sudoku puzzle by generating distinct symbolic integers for the entries in the grid that aren't 1-9.
You can safely ignore the code for now, as we will cover the topics in future tutorials, or you can check out the documentation for [`GenSym`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:GenSym) and [`Fresh`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Fresh).

Here we also used the pretty printer shipped with Grisette, with the [`PPrint`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:PPrint) class. It is based on [`prettyprinter`](https://hackage.haskell.org/package/prettyprinter), and we additionally provided operator precedence handling and generic derivation.

In [22]:
genSymBoard :: Identifier -> [[Integer]] -> SymBoard
genSymBoard ident  =
    flip runFresh ident
  . (traverse . traverse) (\x -> if x > 0 && x <= 9 then return (con x) else simpleFresh ())

In [23]:
easy :: SymBoard
easy =
  genSymBoard
    "easy"
    [ [0, 0, 0, 0, 0, 0, 0, 8, 0],
      [6, 8, 0, 4, 7, 0, 0, 2, 0],
      [0, 1, 9, 5, 0, 8, 6, 4, 7],
      [0, 6, 0, 9, 0, 0, 0, 0, 4],
      [3, 4, 2, 6, 8, 0, 0, 0, 0],
      [1, 9, 0, 0, 5, 0, 8, 3, 0],
      [0, 0, 0, 7, 2, 0, 4, 0, 3],
      [0, 0, 6, 0, 0, 5, 0, 1, 0],
      [0, 0, 3, 8, 9, 1, 5, 0, 0]
    ]
medium :: SymBoard
medium =
  genSymBoard
    "medium"
    [ [8, 0, 0, 1, 0, 0, 0, 7, 0],
      [0, 2, 0, 0, 4, 0, 8, 0, 0],
      [0, 6, 0, 7, 0, 0, 0, 0, 0],
      [0, 0, 0, 4, 7, 0, 9, 0, 8],
      [2, 4, 0, 0, 8, 0, 0, 0, 0],
      [0, 3, 8, 0, 0, 0, 0, 0, 5],
      [0, 8, 0, 6, 0, 4, 1, 0, 0],
      [9, 0, 0, 0, 0, 7, 2, 0, 4],
      [0, 0, 5, 8, 1, 0, 0, 0, 6]
    ]
hard :: SymBoard
hard =
  genSymBoard
    "hard"
    [ [0, 2, 4, 0, 0, 0, 0, 0, 7],
      [0, 6, 7, 0, 0, 0, 3, 0, 2],
      [0, 0, 0, 0, 4, 0, 5, 0, 0],
      [0, 0, 6, 0, 3, 0, 8, 5, 0],
      [4, 0, 5, 0, 8, 0, 0, 2, 0],
      [0, 0, 0, 0, 0, 0, 0, 7, 0],
      [7, 0, 0, 3, 0, 2, 0, 0, 0],
      [0, 1, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 8, 0, 0, 4, 0, 0, 9]
    ]
expert :: SymBoard
expert =
  genSymBoard
    "expert"
    [ [0, 7, 0, 0, 0, 0, 8, 0, 0],
      [8, 3, 1, 0, 0, 4, 0, 0, 0],
      [0, 4, 0, 0, 0, 0, 0, 0, 0],
      [0, 5, 7, 0, 6, 0, 0, 2, 0],
      [3, 0, 0, 0, 0, 1, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 9, 6, 0],
      [4, 0, 0, 0, 2, 0, 0, 0, 0],
      [0, 0, 0, 3, 0, 0, 6, 0, 5],
      [0, 0, 0, 1, 0, 0, 2, 9, 0]
    ]
-- pformat, provided by PPrint class, is the derivable pretty printer provided by Grisette.
pformat easy

[ [easy@0, easy@1, easy@2, easy@3, easy@4, easy@5, easy@6, 8, easy@7],
  [6, 8, easy@8, 4, 7, easy@9, easy@10, 2, easy@11],
  [easy@12, 1, 9, 5, easy@13, 8, 6, 4, 7],
  [easy@14, 6, easy@15, 9, easy@16, easy@17, easy@18, easy@19, 4],
  [3, 4, 2, 6, 8, easy@20, easy@21, easy@22, easy@23],
  [1, 9, easy@24, easy@25, 5, easy@26, 8, 3, easy@27],
  [easy@28, easy@29, easy@30, 7, 2, easy@31, 4, easy@32, 3],
  [easy@33, easy@34, 6, easy@35, easy@36, 5, easy@37, 1, easy@38],
  [easy@39, easy@40, 3, 8, 9, 1, 5, easy@41, easy@42]
]

### Turning the checker into a solver

After you get your symbolic checker, it is intuitive that we can use a constraint solver to solve the sudoku puzzle.
The symbolic checker generates a symbolic formula such that it evaluates to true when the missing entries are assigned with valid numbers.
Based on this, you can ask the solver to make such assignments.

In [24]:
sudoku :: SymBoard -> IO (Maybe Board)
sudoku puzzle = do
  m <- TODO -- TODO: Call z3 solver to solve the sudoku board
  case m of
    Left _ -> return Nothing
    -- TODO: If the solver finds a solution, use it to evaluate the board and return
    Right model -> return $ Just TODO

: 

The following code calls the sudoku solver.

In [25]:
sudoku easy >>= pprint
sudoku medium >>= pprint
sudoku hard >>= pprint
sudoku expert >>= pprint

: 

### Answer

In [26]:
symCheck :: SymRow -> SymBool
symCheck xs = distinct xs .&& inbound xs
  where
    distinct :: SymRow -> SymBool
    distinct [] = true
    distinct (x : xs1) = symNotElem x xs1 .&& distinct xs1
    inbound :: SymRow -> SymBool
    inbound = symAll (\x -> x .>= 1 .&& x .<= 9)

checkSymBoard :: SymBoard -> SymBool
checkSymBoard rows = symAll symCheck (toCheck rows)

In [27]:
sudoku :: SymBoard -> IO (Maybe Board)
sudoku puzzle = do
  m <- solve z3 $ checkSymBoard puzzle
  case m of
    Left _ -> return Nothing
    Right model -> return $ Just $ evalSymToCon model puzzle

In [28]:
sudoku easy >>= pprint
sudoku medium >>= pprint
sudoku hard >>= pprint
sudoku expert >>= pprint

Just
  [ [7, 3, 4, 1, 6, 2, 9, 8, 5],
    [6, 8, 5, 4, 7, 9, 3, 2, 1],
    [2, 1, 9, 5, 3, 8, 6, 4, 7],
    [5, 6, 8, 9, 1, 3, 2, 7, 4],
    [3, 4, 2, 6, 8, 7, 1, 5, 9],
    [1, 9, 7, 2, 5, 4, 8, 3, 6],
    [8, 5, 1, 7, 2, 6, 4, 9, 3],
    [9, 2, 6, 3, 4, 5, 7, 1, 8],
    [4, 7, 3, 8, 9, 1, 5, 6, 2]
  ]

Just
  [ [8, 9, 4, 1, 3, 5, 6, 7, 2],
    [5, 2, 7, 9, 4, 6, 8, 3, 1],
    [1, 6, 3, 7, 2, 8, 5, 4, 9],
    [6, 5, 1, 4, 7, 3, 9, 2, 8],
    [2, 4, 9, 5, 8, 1, 7, 6, 3],
    [7, 3, 8, 2, 6, 9, 4, 1, 5],
    [3, 8, 2, 6, 9, 4, 1, 5, 7],
    [9, 1, 6, 3, 5, 7, 2, 8, 4],
    [4, 7, 5, 8, 1, 2, 3, 9, 6]
  ]

Just
  [ [5, 2, 4, 8, 6, 3, 1, 9, 7],
    [8, 6, 7, 1, 9, 5, 3, 4, 2],
    [1, 9, 3, 2, 4, 7, 5, 6, 8],
    [2, 7, 6, 4, 3, 9, 8, 5, 1],
    [4, 3, 5, 7, 8, 1, 9, 2, 6],
    [9, 8, 1, 5, 2, 6, 4, 7, 3],
    [7, 4, 9, 3, 1, 2, 6, 8, 5],
    [6, 1, 2, 9, 5, 8, 7, 3, 4],
    [3, 5, 8, 6, 7, 4, 2, 1, 9]
  ]

Just
  [ [6, 7, 2, 5, 1, 3, 8, 4, 9],
    [8, 3, 1, 6, 9, 4, 5, 7, 2],
    [5, 4, 9, 8, 7, 2, 1, 3, 6],
    [1, 5, 7, 4, 6, 9, 3, 2, 8],
    [3, 9, 6, 2, 8, 1, 4, 5, 7],
    [2, 8, 4, 7, 3, 5, 9, 6, 1],
    [4, 1, 5, 9, 2, 6, 7, 8, 3],
    [9, 2, 8, 3, 4, 7, 6, 1, 5],
    [7, 6, 3, 1, 5, 8, 2, 9, 4]
  ]

## Conclusion

In this tutorial, we started a journey into the world of solver-aided programming with Grisette.
We learned how Grisette extends the Haskell programming model with symbolic types, allowing us to represent unknown or uncertain values in our programs.

By introducing symbolic constants and leveraging the power of SMT solvers like Z3, we discovered how to reason about our programs and find concrete values that satisfy given constraints.
We explored the `SymInteger` and `SymBool` types.
These symbolic types serve as the foundation for symbolic reasoning in Grisette.
We then use solvers to get mappings from symbolic constants to concrete values that make our symbolic expressions evaluate to a desired outcome.

To showcase the potential of Grisette, we built a small expression equivalence verifier.
This example demonstrated how we use symbolic execution and SMT solving to prove the equivalence of two expressions or find counterexamples when they are not equivalent.
We then strengthened our understanding by implementing a sudoku puzzle solver.

However, this is just the tip of the iceberg. Grisette is a battery-included system that offers a rich set of features and abstractions that enable us to tackle more complex problems and build sophisticated solver-aided tools.
In future tutorials, we will explore these advanced concepts and learn how to harness the full power of Grisette.

As you continue your journey with Grisette, you will discover a whole new paradigm of programming, where the lines between traditional development and formal methods blur.
By embracing symbolic reasoning and solver-aided techniques, you will unlock new possibilities for creating reliable, correct, and efficient software.